In [1]:
import pandas as pd
import twitter

In [2]:
consumer_key = ''
consumer_secret = ''
access_token_key = ''
access_token_secret = ''

api = twitter.Api(consumer_key=consumer_key,
                  consumer_secret=consumer_secret,
                  access_token_key=access_token_key,
                  access_token_secret=access_token_secret,
                  sleep_on_rate_limit=True)

print(api.VerifyCredentials())

{"created_at": "Wed Feb 08 17:01:56 +0000 2017", "default_profile": true, "default_profile_image": true, "friends_count": 20, "id": 829374717941448704, "lang": "en", "name": "Dallas Griffin", "profile_background_color": "F5F8FA", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_4_normal.png", "profile_link_color": "1DA1F2", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "screen_name": "DGriff89"}


In [93]:
# Topic is commercials
topic = 'commercial'

# List of Brands with promiment Super Bowl commercials
brands = ['audti','coke','budweiser','ford','mr.%20clean']

# List of Tweet Attributes to Store in Dataframe
attributes = ['id','user.screen_name','created_at','favorite_count','retweet_count','text']

In [94]:
def digest_tweets(twitter_obj,brand):
    '''Function to digest each Twitter Object from the API (typically 100 tweets + metadata) 
    and output a list of dictionaries'''
    
    # Initialize list
    tweet_data = []

    for tweet in twitter_obj:
        d = tweet.__dict__
        tweet_dict = dict((k, d[k]) for k in attributes if k in d)
        tweet_dict['Brand'] = brand
        
        tweet_data.append(tweet_dict)
        
    return tweet_data

In [95]:
def collect_tweets(topic,brands,max_tweets=1000):
    '''Collect tweets on a topic for a list of brands. The count refers 
    to the maximum umber of tweets that will be pulled PER brand'''
    
    tot_tweets_list = [] # Initialize list
    
    for brand in brands:
        
        max_id = 0  # Use min_id to work-around Twitter API's 100 tweet per call limit
        result_len = 100 # Tracker in case there are less then maximum tweets available.
        
        brand_count = max_tweets
        
        while brand_count >= 100 and result_len == 100:
            
            if max_id == 0:
                twitter_obj = api.GetSearch(raw_query='q='+brand+'%20'+topic+'&lang=en&count=100')
                result_len = len(twitter_obj)
                brand_count -= result_len # Update maximum counter
                max_id = twitter_obj[result_len-1].id # Set maximum ID for next pull as last tweets ID - 1
                
                tweet_list = digest_tweets(twitter_obj,brand) # Outputs list of tweet dicitionaries
                tot_tweets_list += tweet_list
            
            else:
                twitter_obj = api.GetSearch(raw_query='q='+brand+'%20'+topic+'&lang=en&count=100&max_id='+str(max_id))
                result_len = len(twitter_obj)
                brand_count -= result_len # Update maximum counter
                max_id = twitter_obj[result_len-1].id # Set maximum ID for next pull as last tweets ID - 1
                
                tweet_list = digest_tweets(twitter_obj,brand) # Outputs list of tweet dicitionaries
                tot_tweets_list += tweet_list
                
    return tot_tweets_list

In [116]:
twitter_objects_list = collect_tweets(topic,brands,max_tweets=2000)

print "Number of tweets pulled:" + str(len(twitter_objects_list))

Number of tweets pulled:5135


In [117]:
# Create DataFrame of Tweets
tweets_df = pd.DataFrame(twitter_objects_list)
tweets_df = tweets_df.set_index('id')

# Fix unicode issue with text
tweets_df['text'] = tweets_df['text'].apply(lambda row: row.strip().encode('utf-8'))

tweets_df.head()

,Brand,created_at,favorite_count,retweet_count,text
id,,,,,
829484931701145600,audi,Thu Feb 09 00:19:53 +0000 2017,0,0,Great brand positioning! #Audi Super Bowl ad m...
829484460286603265,audi,Thu Feb 09 00:18:01 +0000 2017,0,0,That #Audi commercial is so awful.
829481014141517826,audi,Thu Feb 09 00:04:19 +0000 2017,0,0,"Watch ""Parody Audi Super Bowl Commercial"" on Y..."
829480810956935171,audi,Thu Feb 09 00:03:30 +0000 2017,0,124,RT @USATODAYsports: Audi #SuperBowl commercial...
829478177634410497,audi,Wed Feb 08 23:53:03 +0000 2017,0,0,Whats a woman's worth? Great advertisement for...


In [118]:
tweets_df[['Brand','text']].groupby('Brand').count()

,text
Brand,
Mr.%20Clean,1258
audi,288
budweiser,998
coke,2000
ford,591


In [119]:
# Save to CSV
tweets_df.to_csv('BrandTweets.csv')